In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd
from pyspark.sql.window import Window
import time
from pyspark.sql import SparkSession

/tmp/ipykernel_31780/1510302545.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [5]:
spark = SparkSession.builder \
       .master("local[*]") \
       .appName("test") \
       .config("spark.driver.memory", "80g")\
       .config("spark.executor.memory", "50g")\
       .config("spark.dirver.maxResultSize", "50g")\
       .config('spark.local.dir', '/dados10t/datalake/raw/checkpoint_dir')\
       .getOrCreate()
sc = spark.sparkContext
sc.setCheckpointDir('/dados10t/datalake/raw/checkpoint_dir')

# Srag

In [6]:
#srag = spark.read.parquet("/dados10t/datalake/standard/vigivac/srag/dbsrag_tb_influd_long_09072022_v8.parquet/")
srag = spark.read.parquet("/dados10t/datalake/raw/vigivac/srag/dbsrag_tb_influd_pandas_idrow_09072022.parquet/")

In [7]:
srag = srag.withColumn('srag_bin', F.lit(1))

# SG

In [8]:
sg = spark.read.parquet("/dados10t/datalake/standard/vigivac/SG/sg_long_09072022_v9.parquet/")

In [9]:
sg = sg.withColumn('sg_bin', F.lit(1))

# Vacinas

In [10]:
vac = spark.read.parquet('/dados10t/datalake/standard/vigivac/vacinacao/vacinacao07072022_pessoasunicas_5doses_v17.parquet/')

In [11]:
vac = vac.selectExpr("*",
    "stack(5,'d1_data_aplicacao',d1_data_aplicacao,'d2_data_aplicacao',d2_data_aplicacao,'d3_data_aplicacao',d3_data_aplicacao,'d4_data_aplicacao',d4_data_aplicacao,'d5_data_aplicacao',d5_data_aplicacao) as (n_vacina,data_aplicacao)")\
.filter(F.col('data_aplicacao').isNotNull())

In [9]:
#vac = spark.read.parquet('/dados10t/datalake/raw/vigivac/vacinacao/vacinacao07072022_v5.parquet/')

In [12]:
vac = vac.withColumn('vac_bin', F.lit(1))

# Merge

In [13]:
srag_sg = srag.unionByName(sg,allowMissingColumns=True )

In [16]:
srag_sg_vac = srag_sg.unionByName(vac, allowMissingColumns=True)

In [17]:
full_id = srag_sg_vac.withColumn("new_column", F.lit("ABC"))
w = Window().partitionBy('new_column').orderBy(F.lit('A'))
full_id_with_row = full_id.withColumn("visit_occurrence_id",F.row_number().over(w)).drop("new_column")

In [ ]:
full_id_with_row.columns

In [18]:
full_id_with_row = full_id_with_row.checkpoint()

23/10/19 12:18:37 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


# Writting

### srag

In [19]:
srag_id = full_id_with_row.filter(F.col('srag_bin') == 1).select(srag.columns + ['visit_occurrence_id']).drop('srag_bin')

In [ ]:
srag_id.write.parquet('/dados10t/datalake/standard/omop/data_with_ids/dbsrag_tb_influd_long_09072022_v8-id_omop-v2.parquet')

#### sg

In [ ]:
sg_id = full_id_with_row.filter(F.col('sg_bin') == 1).select(sg.columns + ['visit_occurrence_id']).drop('sg_bin')

In [ ]:
sg_id.write.parquet('/dados10t/datalake/standard/omop/data_with_ids/sg_long_09072022_v9-id_omop-v2.parquet/')

#### vac

In [ ]:
vac_id = full_id_with_row.filter(F.col('vac_bin') == 1).select(vac.columns + ['visit_occurrence_id']).drop('vac_bin')

In [ ]:
vac_id.write.parquet('/dados10t/datalake/standard/omop/data_with_ids/vacinacao07072022_pessoasunicas_5doses_v17-id_omop-v2.parquet/')